In [9]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0  # Switch to EfficientNetB0 for faster training
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
import os

# Mixed Precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Parameters
IMG_SIZE = (128, 128)  # Reduce image size to speed up training
BATCH_SIZE = 32  # You can try lowering the batch size to 16 or 8
EPOCHS = 50
NUM_CLASSES = 5  # 0: Normal, 1: Mild, 2: Moderate, 3: Severe, 4: Proliferative

# Paths to your dataset
train_dir = r"C:\\Users\\ADMIN\\Documents\\Project updated\\aptos-augmented-images\\train\\retina"

# Preprocessing function (Ben Graham's method)
def preprocess_image(image):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2,
    preprocessing_function=preprocess_image
)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Use EfficientNetB0 for faster training
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with Adam optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(
    'best_dr_grading_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# Initial training with frozen layers
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

# Save the final model in .h5 format
model.save('dr_grading_final.h5')

# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=1)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


Found 8000 images belonging to 5 classes.
Found 2000 images belonging to 5 classes.
Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1930 - loss: 1.6713
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to best_dr_grading_model.h5


250/250 ━━━━━━━━━━━━━━━━━━━━ 1451s 6s/step - accuracy: 0.1930 - loss: 1.6711 - val_accuracy: 0.2000 - val_loss: 1.6094 - learning_rate: 0.0010
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1981 - loss: 1.6096
Epoch 2: val_accuracy did not improve from 0.20000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1538s 6s/step - accuracy: 0.1981 - loss: 1.6096 - val_accuracy: 0.2000 - val_loss: 1.6094 - learning_rate: 0.0010
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.1950 - loss: 1.6097
Epoch 3: val_accuracy did not improve from 0.20000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1469s 6s/step - accuracy: 0.1950 - loss: 1.6097 - val_accuracy: 0.2000 - val_loss: 1.6094 - learning_rate: 0.0010
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2037 - loss: 1.6095
Epoch 4: val_accuracy did not improve from 0.20000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1352s 5s/step - accuracy: 0.2037 - loss: 1.6095 - val_accuracy: 0.2000 - val_loss: 1.6094 - learning_rate: 0.0010
Epoch 5/50
250/250 ━

KeyboardInterrupt: 